## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kuryk,KZ,43.2000,51.6500,45.48,84,81,15.86,broken clouds
1,1,Iquique,CL,-20.2208,-70.1431,75.24,64,3,11.61,clear sky
2,2,Hobart,AU,-42.8794,147.3294,57.25,77,75,8.05,broken clouds
3,3,Dubbo,AU,-32.2500,148.6167,55.53,94,0,9.22,clear sky
4,4,Labuhan,ID,-6.8844,112.2051,78.13,80,100,4.18,overcast clouds


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 60
What is the maximun temperature you would like for your trip? 100


In [31]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Iquique,CL,-20.2208,-70.1431,75.24,64,3,11.61,clear sky
4,4,Labuhan,ID,-6.8844,112.2051,78.13,80,100,4.18,overcast clouds
5,5,Ndungu,TZ,-4.3667,38.0500,72.43,92,100,3.04,light rain
7,7,Meulaboh,ID,4.1363,96.1285,77.63,84,96,3.91,overcast clouds
9,9,Durango,MX,24.8333,-104.8333,63.01,12,1,11.97,clear sky


In [32]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

283


City_ID                283
City                   283
Country                281
Lat                    283
Lng                    283
Max Temp               283
Humidity               283
Cloudiness             283
Wind Speed             283
Current Description    283
dtype: int64

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

281


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Iquique,CL,-20.2208,-70.1431,75.24,64,3,11.61,clear sky
4,4,Labuhan,ID,-6.8844,112.2051,78.13,80,100,4.18,overcast clouds
5,5,Ndungu,TZ,-4.3667,38.0500,72.43,92,100,3.04,light rain
7,7,Meulaboh,ID,4.1363,96.1285,77.63,84,96,3.91,overcast clouds
9,9,Durango,MX,24.8333,-104.8333,63.01,12,1,11.97,clear sky


In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Iquique,CL,75.24,clear sky,-20.2208,-70.1431,
4,Labuhan,ID,78.13,overcast clouds,-6.8844,112.2051,
5,Ndungu,TZ,72.43,light rain,-4.3667,38.0500,
7,Meulaboh,ID,77.63,overcast clouds,4.1363,96.1285,
9,Durango,MX,63.01,clear sky,24.8333,-104.8333,
12,Kokopo,PG,76.01,overcast clouds,-4.3520,152.2633,
13,Gamba,GA,78.53,light rain,-2.6500,10.0000,
14,Atuona,PF,77.68,light rain,-9.8000,-139.0333,
16,Hilo,US,77.11,clear sky,19.7297,-155.0900,
17,Fortuna,US,66.92,clear sky,40.5982,-124.1573,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Iquique,CL,75.24,clear sky,-20.2208,-70.1431,Backpacker`s Hostel Iquique
4,Labuhan,ID,78.13,overcast clouds,-6.8844,112.2051,DoubleFAG
5,Ndungu,TZ,72.43,light rain,-4.3667,38.0500,Nick Resort
7,Meulaboh,ID,77.63,overcast clouds,4.1363,96.1285,Kost Pria
12,Kokopo,PG,76.01,overcast clouds,-4.3520,152.2633,Kokopo Beach Bungalow Resort
14,Atuona,PF,77.68,light rain,-9.8000,-139.0333,Villa Enata
16,Hilo,US,77.11,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Fortuna,US,66.92,clear sky,40.5982,-124.1573,Super 8 by Wyndham Fortuna
18,Tabou,CI,78.58,clear sky,4.4230,-7.3528,hotêl doufoulougou
19,Kavieng,PG,80.58,overcast clouds,-2.5744,150.7967,Nusa Island Retreat


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))